앞서 \ref{chap:TyChk}장에서 살펴본 FACT언어의 문법구조(\ref{fig:FACTlang})에서는
함수요약식과 재귀함수요약식에서 유효범휘에 묶이는 이름의 타입이 무엇인지 항상 표기해야 한다.
이렇듯 함수 파라메터를 비롯한 새로운 이름이 유효범위에 묶일 때마다
타입 정보를 프로그래머가 제공하도록 강제하는 언어에서
타입유추를 통해 알아내는 주요한 정보는 바로 함수가 계산하는 결과값의 타입이다.
그런데, 우리가 이 책으로 프로그래밍언어 이론를 배우며 실습용으로 활용하는 하스켈의 경우에는
람다식이나 함수 선언을 할 때 함수 파라메터에 타입을 표기하지 않아도 많은 경우 타입유추가 
가능하다는 것을 직접 경험했을 것이다. 이번 장에서는 FACT언어를 사용하는 프로그래머가
타입 정보를 덜 제공하더라도 타입을 자동으로 알아낼 수 있도록 더 많은 타입유추 기능을 추가해 보자.

In [1]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

\newpage
# FACT언어에서 타입 표기의 생략 허용
앞서 \ref{chap:TyChk}장에서 살펴본 FACT언어의 문법구조(\ref{fig:FACTlang})에서는
함수요약식과 재귀함수요약식에서 유효범휘에 묶이는 이름의 타입이 무엇인지 항상 나타난다.
우선, 함수요약식과 재귀함수요약식에서 타입 정보가 생략된 경우를 허용하도록,
즉 $\lambda x.e$나 $\textbf{rec}\,f\,(\lambda\cdots)$와 같은 형태도 허용하도록,
지난 \ref{chap:TyChk}장에서 FACT언어의 문법구조를 표현하던 `Expr`을
아래와 같이 조금 수정하였다. 수정된 부분은 `Lam`과 `Rec`의
선언 부분에서 기존의 `Ty` 대신 나타나는 `Maybe Ty`이다.

In [2]:
type Nm = String  -- 변수 이름은 문자열로

data Ty = Z | Arr Ty Ty  deriving (Eq,Ord)

data Expr = Var Nm                 -- x
          | Lam Nm (Maybe Ty) Expr -- (\x:t.e)
          | Rec Nm (Maybe Ty) Expr -- rec f:t (\x:t2.e)
          | App Expr Expr          -- (e1 e2)
          | Lit Int                -- n
          | Add Expr Expr          -- e1 + e2
          | If Expr Expr Expr      -- if e then e1 else e0
          deriving (Eq, Ord)

data Val = Cl Expr Env | Vi Int  deriving (Eq,Ord)
type Env = [ (Nm, Val) ]
type TEnv = [ (Nm, Ty) ]

In [3]:
instance Show Ty where
  showsPrec _ Z = showString "\\mathbb{Z}"
  showsPrec p (Arr t1 t2) = showParen (p > 1) $
      showsPrec p t1 . showString "{\\to}" . showsPrec p t2
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x mt e) = showParen (p > 1) $
      showString ("\\lambda "++x) . showsMT mt .
      showString "." . showsPrec 1 e
  showsPrec p (Rec f mt e) = showParen (p > 9) $
      showString ("\\textbf{rec}\\,"++f) . showsMT mt .
      showString "\\, " . showsPrec 2 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
  showsPrec _ (Lit n) = shows n
  showsPrec p (Add e1 e2) = showParen (p > 6) $
      showsPrec 6 e1 . showString "+" . showsPrec 7 e2
  showsPrec p (If e e1 e0) = showParen (p > 2) $
      showString "\\textbf{if}\\;" . showsPrec 3 e .
      showString "\\;\\textbf{then}\\;" . showsPrec 3 e1 .
      showString "\\;\\textbf{else}\\;" . showsPrec 3 e0

showsMT (Just t) = showString "{:}" . showsPrec 0 t
showsMT Nothing  = id 

import IHaskell.Display (latex)
instance IHaskellDisplay Expr where -- 노트북에 디스플레이하는 인스턴스 선언
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Ty where
    display t = display [latex $ "$"++show t++"$"]

In [4]:
idExpr = Lam "x" Nothing (Var "x")
ttExpr = Lam "x" (Just Z) (Lam "y" (Just Z) (Var "x")) 
ffExpr = Lam "x" (Just Z) (Lam "y" (Just Z) (Var "y")) 
-- 일부 이름 바인딩에 타입 표기가 생략된 예시 프로그램
If (Add (Lit 3) (Var "y"))  ffExpr
                            (App idExpr (Rec "f" Nothing ffExpr))

In [5]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show TEnv where
    show tenv = "\\{"++ intercalate ",\\," (map show tenv) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Val) where
    show (x,v) = x++"{\\mapsto}"++show v
instance {-# OVERLAPS #-} Show (Nm,Ty) where
    show (x,t) = x++"{\\mapsto}"++show t
instance Show Val where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "
    show (Vi n) = " "++show n++" "

instance {-# OVERLAPS #-} IHaskellDisplay Env where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay TEnv where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Val) where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Ty) where
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Val where
    display v = display [latex $ "$"++show v++"$"]

타입검사 함수 `check`와 타입유추 함수 `infer`는 기존의 내용을 사실상 그대로 재사용할 것이다.
타입 표기가 전혀 생략되지 않은 식, 즉 `Nothing`이 하나도 나타나지 않는 식에 대해서만
`check`의 결과가 참(`True`)이 될 수 있으며 `infer`의 결과도 비어있지 않고 유추된 타입이
들어있는 리스트(`[`$\tau\!\!$ `]`)가 될 수 있다.

In [19]:
check tenv e t = infer tenv e == [t]  -- check는 infer로 간단히 정의 가능

infer :: TEnv -> Expr -> [Ty]  -- 적절한 타입이 존재하지 않으면 빈 리스트
infer tenv (Var x)      = case lookup x tenv of Just t  -> [t]
                                                Nothing -> []
infer tenv (Lam x (Just t2) e) = undefined
infer tenv (Lam x Nothing   e) = []  -- 생략된 타입 표기가 있으면 실패
infer tenv (Rec f (Just t) e)  = undefined
infer tenv (Rec f Nothing  e)  = []  -- 생략된 타입 표기가 있으면 실패
infer tenv (App e1 e2)  = undefined
infer tevv (Lit _)      = [Z]
infer tenv (Add e1 e2)  = [Z | check tenv e1 Z, check tenv e2 Z]
infer tenv (If e e1 e0) = undefined

\ref{chap:TyChk}장에서와 마찬가지 내용으로 위의 `infer`와 아래의 `ev` 함수에서
`undefined`로 처리된 부분을 작성하면 된다.

인터프리터 함수 `ev` 및 `eval'`도 기존의 내용을 그대로 재사용한다.
아래의 인터프리터 함수 `ev`는 타입유추가 성공한 경우, 즉 `infer`의
결과가 빈 리스트가 아닌 경우에만 실행한다고 가정한다.

In [20]:
ev :: Env -> Expr -> Val  -- infer가 성공한 경우에만 실행함을 가정
ev env (Var x) = v
    where Just v = lookup x env
ev env e@(Lam _ _ _) = Cl e env
ev env (Rec f t e@(Lam _ _ _)) = undefined -- 적절히 작성
ev env (App e1 e2) = ev ((x,v2):env1) e
    where Cl (Lam x _ e) env1 = ev env e1 
          v2                  = ev env e2 
ev _   (Lit n)     = Vi n
ev env (Add e1 e2) = Vi (n1 + n2)
    where Vi n1 = ev env e1
          Vi n2 = ev env e2
ev env (If e e1 e0) = if n/=0 then ev env e1
                              else ev env e0
    where Vi n = ev env e

In [21]:
-- infer에 성공한 닫힌식에 대한 인터프리터
eval' e = [ev [] e | t <- infer [] e]

0부터 주어진 자연수 $i$까지의 총합을 구하는 재귀함수를 다음과 같이 작성할 수 있다.

In [22]:
e4 = Rec "f" (Just $ Arr Z Z) .
         Lam "i" (Just Z) $ If i (i `Add` App f (i `Add` Lit (-1))) i
    where f = Var "f"
          i = Var "i"
e4 -- rec f (\i.if i then i + f(i + -1) else i)

`infer`와 `ev`함수의 `undefined` 부분을 제대로 채워넣었다면
아래에 보이는 에러 없이 정상적으로 0부터 100까지의 합 5050이 계산될 것이다.

In [23]:
-- rec f (\i.if i then i + f(i + -1) else i) 100
eval' (e4 `App` Lit 100) -- 100부터 0까지의 총합

: 

\section*{요점정리}
* TODO
* TODO

\section*{연습문제}
1. TODO
1. TODO

\section*{탐구과제}
1. TODO aaa TODO
1. TODO aaa TODO